In [1]:
!pip install transformers


In [2]:
import sys
import os

module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from captions_dataset import CaptionsDataset

dataset_train = CaptionsDataset(split="train")
dataset_val = CaptionsDataset(split="val")

Dataset json file, loading dataset...
There are 118287 captions in total (train)
The following 91 categories are present in the dataset:
['bicycle', 'vehicle', 'clock', 'indoor', 'cup', 'kitchen', 'sink', 'appliance', 'vase', 'car', 'motorcycle', 'person', 'bench', 'outdoor', 'airplane', 'bottle', 'toilet', 'furniture', 'potted plant', 'toothbrush', 'refrigerator', 'oven', 'apple', 'food', 'banana', 'surfboard', 'sports', 'bowl', 'spoon', 'traffic light', 'cat', 'animal', 'cow', 'handbag', 'accessory', 'umbrella', 'book', 'skateboard', 'horse', 'cake', 'donut', 'chair', 'cell phone', 'electronic', 'laptop', 'mouse', 'tv', 'dog', 'knife', 'orange', 'microwave', 'backpack', 'bus', 'dining table', 'truck', 'bird', 'giraffe', 'suitcase', 'boat', 'skis', 'fire hydrant', 'toaster', 'wine glass', 'sheep', 'remote', 'scissors', 'tie', 'kite', 'broccoli', 'stop sign', 'teddy bear', 'fork', 'keyboard', 'elephant', 'carrot', 'parking meter', 'snowboard', 'pizza', 'baseball glove', 'frisbee', 'tra

In [3]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
print("Tokenizer before added special tokens "+str(len(tokenizer)))
special_tokens_dict = {'additional_special_tokens': dataset_train.control_codes}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print("added "+str(num_added_toks)+" tokens to the pretrained tokenizer")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Tokenizer before added special tokens 50259
added 91 tokens to the pretrained tokenizer


Embedding(50350, 768)

In [4]:
dataset_train.tokenize(tokenizer, load_tokenized=True)
dataset_val.tokenize(tokenizer, load_tokenized=True)

Loading pretokenized dataset! (train)
Tokenizing dataset! (val)
Saving tokenized dataset! (val)


In [5]:
import random
import torch
import numpy as np

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=dataset_train,         # training dataset
    eval_dataset=dataset_val)

In [8]:
trainer.train()

ValueError: num_samples should be a positive integer value, but got num_samples=0